# Lab 10.5.1: Advanced CNN (VGG)

Edited By Steve Ive

Here, we are going to learn about the advanced CNN, which is called as 'VGG', made by Oxford.
More about the VGG net, please read the article before this '10.5.0 About the VGG net.md'

Reference From

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_5_1_Advance-cnn(VGG).ipynb

## Follow Codes are same as

```import torchvision.models.vgg```

In [10]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [11]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

### Take a Moment!

```torch.nn.init.normal_(tensor, mean=0.0, std=1.0)```

Fills the input Tensor with values drawn from the normal distribution $\mathcal{N}(\text{mean}, \text{std}^2)$.

**Parameters**

> - **tensor** – an n-dimensional torch.Tensor

> - **mean** – the mean of the normal distribution

> - **std**  – the standard deviation of the normal distribution


```torch.nn.init.constant_(tensor, val)```

Fills the input Tensor with the value $\text{val}$.

**Parameters**

> - **tensor** – an n-dimensional torch.Tensor

> - **val** – the value to fill the tensor with

```torch.nn.init.xavier_normal_(tensor, gain=1.0)```

Fills the input Tensor with values according to the method described in Understanding the difficulty of training deep feedforward neural networks - Glorot, X. & Bengio, Y. (2010), using a normal distribution. The resulting tensor will have values sampled from $\mathcal{N}(0, \text{std}^2)$ where

$\text{std} = \text{gain} \times \sqrt{\frac{2}{\text{fan\_in} + \text{fan\_out}}}$
​
 
Also known as Glorot initialization.

**Parameters**

> - **tensor** – an n-dimensional torch.Tensor

> - **gain** – an optional scaling factor


```torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')```

Fills the input Tensor with values according to the method described in Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification - He, K. et al. (2015), using a normal distribution. The resulting tensor will have values sampled from $\mathcal{N}(0, \text{std}^2)$ where

$\text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}$
​
 
Also known as He initialization.

**Parameters**

> - **tensor** – an n-dimensional torch.Tensor

> - **a** – the negative slope of the rectifier used after this layer (only used with 'leaky_relu')

> - **mode** – either 'fan_in' (default) or 'fan_out'. Choosing 'fan_in' preserves the magnitude of the variance of the weights in the forward pass. Choosing 'fan_out' preserves the magnitudes in the backwards pass.

> - **nonlinearity** – the non-linear function (nn.functional name), recommended to use only with 'relu' or 'leaky_relu' (default).

In [28]:
class VGG(nn.Module):
    def __init__(self, features, num_classes = 1000, init_weights = True):
        super(VGG, self).__init__()

        self.features = features #convolution

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )#FC Layers

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) #Convolution
        x = self.avgpool(x) #avgpool
        x = x.view(x.size(0), -1)
        x = self.classifer(x) #FC Layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [29]:
def make_layers(cfg, batch_norm = False):
    layers = []
    in_channels = 3

    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size = 3, padding = 1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace = True)]
            else:
                layers += [conv2d, nn.ReLU(inplace = True)]
            in_channels = v
    
    return nn.Sequential(*layers)

In [30]:
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
}

In [31]:
conv = make_layers(cfg['A'], batch_norm=True)

In [32]:
CNN = VGG(conv, num_classes = 10, init_weights = True)

In [33]:
CNN

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke